In [15]:
!pip install py7zr

In [16]:
import pandas as pd
import numpy as np
import py7zr
import os
import json
from urllib.request import urlopen

In [17]:
from google.colab import drive
drive.mount("/content/gdrive")

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [18]:
# Data reading
PATH = "/content/gdrive/MyDrive/GlobalTerrorismViz/"

In [19]:
# Data reading
with py7zr.SevenZipFile(PATH + 'data/globalterrorismdb_0221dist.7z') as z:
    # open the csv file in the dataset
    targetPath = PATH + 'data/'
    z.extract(path = targetPath)
    df = pd.read_excel(targetPath + '/globalterrorismdb_0221dist.xlsx')
    
# We filtered out doubted attacks to be able to have exact attacks
df = df[(df['doubtterr'] != 1) & (df['iyear'] >=2000)]

_df = df[['eventid','iyear','imonth','iday','country','country_txt',
         'region','region_txt', 'provstate', 'city','latitude','longitude',
         'nkill', 'nwound','gname','targtype1_txt']]

In [20]:
# Geojson for polygons of map
with urlopen('https://raw.githubusercontent.com/johan/world.geo.json/master/countries.geo.json') as response:
    countries = json.load(response)
                          
countries_df = pd.json_normalize(countries,  record_path =['features'])

In [21]:
# Merging locations and main df
df_merged = pd.merge(_df, countries_df , left_on = 'country_txt', right_on = 'properties.name', how = 'left' )

# Getting Stats
group_year_dataset = df_merged.groupby(by=['gname','iyear','country_txt','id']).agg({'eventid':'nunique',
                                                                'nkill':'sum',
                                                                'nwound':'sum'}).reset_index()

attacks_by_target = df_merged.groupby(by=['targtype1_txt','iyear','country_txt','id']).agg({'eventid':'nunique',
                                                                'nkill':'sum',
                                                                'nwound':'sum'}).reset_index()

death_injury_yearly = df_merged.groupby(by=['iyear','country_txt','id']).agg({'eventid':'nunique',
                                                                'nkill':'sum',
                                                                'nwound':'sum'}).reset_index()

df_merged['affiliation'] = df_merged['gname'].apply(lambda x: 'Affiliated' if x !='Unknown' else 'NotAffiliated')
affiliated_yearly = df_merged.groupby(by=['iyear','country_txt','id','affiliation']).agg({'eventid':'nunique',
                                                                              'nkill':'sum',
                                                                               'nwound':'sum'}).reset_index()

In [27]:
# Writing to excel
group_year_dataset.to_excel(PATH + 'data/xlsx/group_year_dataset.xlsx', engine='openpyxl', encoding='utf-8')
attacks_by_target.to_excel(PATH + 'data/xlsx/attacks_by_target.xlsx', engine='openpyxl', encoding='utf-8')
death_injury_yearly.to_excel(PATH + 'data/xlsx/death_injury_yearly.xlsx', engine='openpyxl', encoding='utf-8')
affiliated_yearly.to_excel(PATH + 'data/xlsx/affiliated_yearly.xlsx', engine='openpyxl', encoding='utf-8')

In [28]:
# Writing to JSON
group_year_dataset.to_json(PATH + 'data/json/group_year_dataset.json')
attacks_by_target.to_json(PATH + 'data/json/attacks_by_target.json')
death_injury_yearly.to_json(PATH + 'data/json/death_injury_yearly.json')
affiliated_yearly.to_json(PATH + 'data/json/affiliated_yearly.json')